# Cityscape Dataset

In [6]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import cv2
import numpy as np
import tqdm
from cityscape_dataset import Cityscape
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
s = set()
ori_gt = []
for root, dirs, files in os.walk("./training/cityscapes/gtFine/train/"):
    for file in files:
        if file.endswith("_gtFine_labelIds.png"):
            ori_gt.append(os.path.join(root, file))

for gt in tqdm.tqdm(ori_gt):
    im = cv2.imread(gt)
    im = im[:, :, 0]
    for i in range(im.shape[0]):
        for j in range(im.shape[1]):
            s.add(im[i][j])

s

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


KeyboardInterrupt: 

In [7]:
from share import *
import config

import cv2
import einops
import numpy as np
import torch
import random

from pytorch_lightning import seed_everything
from annotator.util import resize_image, HWC3
from annotator.uniformer import UniformerDetector
from cldm.model import create_model, load_state_dict
from cldm.ddim_hacked import DDIMSampler


apply_uniformer = UniformerDetector()

model = create_model('./models/cldm_v15.yaml').cpu()
model.load_state_dict(load_state_dict('./results/cityscapes/23-06-27/lightning_logs/version_0/checkpoints/epoch=40-step=30503.ckpt', location='cuda'))
model = model.cuda()
ddim_sampler = DDIMSampler(model)

def process(input_image, prompt, a_prompt, n_prompt, num_samples, image_resolution, detect_resolution, ddim_steps, guess_mode, strength, scale, seed, eta):
    with torch.no_grad():
        input_image = HWC3(input_image)
        # detected_map = apply_uniformer(resize_image(input_image, detect_resolution))
        detected_map = resize_image(input_image, detect_resolution)
        img = resize_image(input_image, image_resolution)
        H, W, C = img.shape

        detected_map = cv2.resize(detected_map, (W, H), interpolation=cv2.INTER_NEAREST)

        control = torch.from_numpy(detected_map.copy()).float().cuda() / 255.0
        control = torch.stack([control for _ in range(num_samples)], dim=0)
        control = einops.rearrange(control, 'b h w c -> b c h w').clone()

        if seed == -1:
            seed = random.randint(0, 65535)
        seed_everything(seed)

        if config.save_memory:
            model.low_vram_shift(is_diffusing=False)

        cond = {"c_concat": [control], "c_crossattn": [model.get_learned_conditioning([prompt + ', ' + a_prompt] * num_samples)]}
        un_cond = {"c_concat": None if guess_mode else [control], "c_crossattn": [model.get_learned_conditioning([n_prompt] * num_samples)]}
        shape = (4, H // 8, W // 8)

        if config.save_memory:
            model.low_vram_shift(is_diffusing=True)

        model.control_scales = [strength * (0.825 ** float(12 - i)) for i in range(13)] if guess_mode else ([strength] * 13)  # Magic number. IDK why. Perhaps because 0.825**12<0.01 but 0.826**12>0.01
        samples, intermediates = ddim_sampler.sample(ddim_steps, num_samples,
                                                     shape, cond, verbose=False, eta=eta,
                                                     unconditional_guidance_scale=scale,
                                                     unconditional_conditioning=un_cond)

        if config.save_memory:
            model.low_vram_shift(is_diffusing=False)

        x_samples = model.decode_first_stage(samples)
        x_samples = (einops.rearrange(x_samples, 'b c h w -> b h w c') * 127.5 + 127.5).cpu().numpy().clip(0, 255).astype(np.uint8)

        results = [x_samples[i] for i in range(num_samples)]
    return detected_map, results

Use Checkpoint: False
Checkpoint Number: [0, 0, 0, 0]
Use global window for all blocks in stage3
load checkpoint from local path: /data0/tangkaiping/workspaces/research/ControlNet/annotator/ckpts/upernet_global_small.pth
ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Loaded model config from [./models/cldm_v15.yaml]
Loaded state_dict from [./results/cityscapes/23-06-27/lightning_logs/version_0/checkpoints/epoch=40-step=30503.ckpt]


In [10]:
import cv2
import albumentations as A
img_file = "training/cityscapes/gtFine/train/aachen/aachen_000068_000019_gtFine_color.png"

fix_prompt = "a professional, detailed, high-quality image of a cityscape"

transform = A.Compose([
    A.Resize(width=1536, height=768),
    A.RandomCrop(width=768, height=768),
    A.HorizontalFlip(p=0.5)
])

image = cv2.imread(img_file)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = transform(image=image)['image']
# cv2.imwrite("_image.png", image)

control, result = process(image, fix_prompt, "", "", 1, 768, 768, 100, False, 1.0, 1.0, 16243, 0.0)

Global seed set to 16243


Data shape for DDIM sampling is (1, 4, 96, 96), eta 0.0
Running DDIM Sampling with 100 timesteps


DDIM Sampler: 100%|██████████| 100/100 [00:38<00:00,  2.62it/s]


In [11]:
cv2.imwrite("_control.png", control)
cv2.imwrite("_result.png", result[0])

True

: 